In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np
import torch.optim as optim


C:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


### 搭建模型

In [2]:
class LinearBNAC(nn.Module):
    def __init__(self, in_channels, out_channels, bias=True, dropout=0.3, is_output=False):
        super(LinearBNAC, self).__init__()
        if is_output and out_channels==1:
            self.linear = nn.Sequential(
                nn.Linear(in_channels, out_channels, bias=bias),
                nn.Sigmoid()
            )
        elif is_output:
            self.linear = nn.Sequential(
                nn.Linear(in_channels, out_channels, bias=bias),
                nn.Softmax(dim=1)
            )   
        else:
            self.linear = nn.Sequential(
                nn.Linear(in_channels, out_channels, bias=bias),
                nn.Dropout(dropout),
                nn.BatchNorm1d(out_channels),
                nn.LeakyReLU(inplace=True)
            )
            
    def forward(self, x):
        out=self.linear(x)
        return out

In [3]:
class Model(nn.Module):
    def __init__(self, input_dimention, output_classes=1):
        super(Model, self).__init__()
        self.layer1 = LinearBNAC(input_dimention, 128)
        self.layer2 = LinearBNAC(128,64) #"自行定義，只要確定模型能順利運行即可，參數值沒有一定限制"
        self.layer3 = LinearBNAC(64,32)   #"自行定義，只要確定模型能順利運行即可，參數值沒有一定限制"
        self.output = LinearBNAC(32, output_classes, is_output=True)
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.output(x)
        return x 
        

### 準備輸入資料、優化器、標籤資料、模型輸出

In [4]:
model = Model(input_dimention=256,output_classes=10)
optimizer = optim.Adam(params=model.parameters(), lr=1e-3, weight_decay=1e-3) #"使用Adam optimizer"

In [5]:
batch_size = 4
input_features = 256
dummy_input = torch.randn(batch_size, input_features,)

#target = torch.empty(4, dtype=torch.float).random_(10)
target = torch.tensor([9., 5., 4., 4.], dtype=torch.long)

In [6]:
output = model(dummy_input) #"自行輸入"
print(output)

tensor([[0.1281, 0.0298, 0.1143, 0.0676, 0.1014, 0.0749, 0.1000, 0.1726, 0.1084,
         0.1028],
        [0.0918, 0.0786, 0.0843, 0.0825, 0.2176, 0.1039, 0.0522, 0.1460, 0.0463,
         0.0967],
        [0.0985, 0.0626, 0.0813, 0.0393, 0.1698, 0.0804, 0.0773, 0.1298, 0.1414,
         0.1196],
        [0.0827, 0.0855, 0.1209, 0.0867, 0.1212, 0.0692, 0.0545, 0.1845, 0.1464,
         0.0483]], grad_fn=<SoftmaxBackward>)


### 計算 CrossEntropy Loss
* 請注意哪一個 Loss最適合：我們已經使用 softmax
* 因為我們有使用dropout，並隨機產生dummy_input，所以各為學員得到的值會與解答不同，然而步驟原理需要相同

In [7]:
from torch.nn import NLLLoss, LogSoftmax, CrossEntropyLoss

In [8]:
criterion = NLLLoss() #"自行輸入"

In [9]:
loss = criterion(torch.log(output), target)
loss

tensor(2.1055, grad_fn=<NllLossBackward>)

### 完成back propagation並更新梯度

In [10]:
#"自行輸入"
loss.backward()

In [11]:
print('weight : {}'.format(model.layer1.linear[0].weight))
print('\n')
print('grad : {}'.format(model.layer1.linear[0].weight.grad))

weight : Parameter containing:
tensor([[-0.0328,  0.0061, -0.0109,  ..., -0.0024,  0.0199, -0.0413],
        [-0.0507,  0.0474,  0.0406,  ...,  0.0329,  0.0588, -0.0184],
        [-0.0202, -0.0311, -0.0160,  ...,  0.0602,  0.0303, -0.0624],
        ...,
        [ 0.0170,  0.0282,  0.0563,  ..., -0.0240,  0.0032,  0.0020],
        [ 0.0507, -0.0617,  0.0491,  ...,  0.0297, -0.0535,  0.0485],
        [-0.0525,  0.0109,  0.0463,  ...,  0.0157, -0.0084, -0.0477]],
       requires_grad=True)


grad : tensor([[-0.0077,  0.0035, -0.0011,  ...,  0.0050, -0.0044,  0.0030],
        [ 0.0032,  0.0204, -0.0332,  ...,  0.0046,  0.0017, -0.0173],
        [ 0.0032,  0.0084, -0.0048,  ...,  0.0200, -0.0048,  0.0100],
        ...,
        [ 0.0017, -0.0028,  0.0030,  ..., -0.0028,  0.0044, -0.0007],
        [ 0.0109, -0.0060,  0.0033,  ..., -0.0076,  0.0130, -0.0049],
        [-0.0357, -0.0837, -0.0157,  ..., -0.2921, -0.2243, -0.1376]])


In [12]:
"自行輸入"
optimizer.step()  # 使用 optimizer.step() 更新參數

In [13]:
print('weight : {}'.format(model.layer1.linear[0].weight))
print('\n')
print('grad : {}'.format(model.layer1.linear[0].weight.grad))

weight : Parameter containing:
tensor([[-0.0318,  0.0051, -0.0099,  ..., -0.0034,  0.0209, -0.0423],
        [-0.0517,  0.0464,  0.0416,  ...,  0.0319,  0.0578, -0.0174],
        [-0.0212, -0.0321, -0.0150,  ...,  0.0592,  0.0313, -0.0634],
        ...,
        [ 0.0160,  0.0292,  0.0553,  ..., -0.0230,  0.0022,  0.0030],
        [ 0.0497, -0.0607,  0.0481,  ...,  0.0307, -0.0545,  0.0495],
        [-0.0515,  0.0119,  0.0473,  ...,  0.0167, -0.0074, -0.0467]],
       requires_grad=True)


grad : tensor([[-0.0078,  0.0035, -0.0011,  ...,  0.0050, -0.0044,  0.0029],
        [ 0.0031,  0.0204, -0.0331,  ...,  0.0046,  0.0017, -0.0173],
        [ 0.0032,  0.0084, -0.0048,  ...,  0.0201, -0.0048,  0.0099],
        ...,
        [ 0.0017, -0.0028,  0.0031,  ..., -0.0029,  0.0044, -0.0007],
        [ 0.0110, -0.0060,  0.0034,  ..., -0.0076,  0.0130, -0.0048],
        [-0.0358, -0.0837, -0.0156,  ..., -0.2921, -0.2243, -0.1377]])


### 清空 gradient

In [14]:
"自行輸入"
optimizer.zero_grad() # 清理目前參數上所殘留的梯度，並開始下一個 iteration 的計算

In [15]:
print('weight : {}'.format(model.layer1.linear[0].weight))
print('\n')
print('grad : {}'.format(model.layer1.linear[0].weight.grad))

weight : Parameter containing:
tensor([[-0.0318,  0.0051, -0.0099,  ..., -0.0034,  0.0209, -0.0423],
        [-0.0517,  0.0464,  0.0416,  ...,  0.0319,  0.0578, -0.0174],
        [-0.0212, -0.0321, -0.0150,  ...,  0.0592,  0.0313, -0.0634],
        ...,
        [ 0.0160,  0.0292,  0.0553,  ..., -0.0230,  0.0022,  0.0030],
        [ 0.0497, -0.0607,  0.0481,  ...,  0.0307, -0.0545,  0.0495],
        [-0.0515,  0.0119,  0.0473,  ...,  0.0167, -0.0074, -0.0467]],
       requires_grad=True)


grad : tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
